In [2]:
import habitat_sim

import random
%matplotlib inline
import matplotlib.pyplot as plt
import PyQt5
import os

import numpy as np
import json

In [3]:
test_scene = "/media/shubodh/DATA/Downloads/data-non-onedrive/replica_v1/apartment_0/habitat/mesh_semantic.ply"


sim_settings = {
    "width": 1920,  # Spatial resolution of the observations    
    "height": 1080,
    "scene": test_scene,  # Scene path
    "default_agent": 0,  
    "sensor_height": 1.5,  # Height of sensors in meters
    "color_sensor": True,  # RGB sensor
    "semantic_sensor": True,  # Semantic sensor
    "depth_sensor": True,  # Depth sensor
    "seed": 1,
}

# Simulator config

In [4]:
def make_cfg(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.scene.id = settings["scene"]
    # Note: all sensors must have the same resolution
    sensors = {
        "color_sensor": {
            "sensor_type": habitat_sim.SensorType.COLOR,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },
        "depth_sensor": {
            "sensor_type": habitat_sim.SensorType.DEPTH,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },
        "semantic_sensor": {
            "sensor_type": habitat_sim.SensorType.SEMANTIC,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },  
    }
    
    sensor_specs = []
    for sensor_uuid, sensor_params in sensors.items():
        if settings[sensor_uuid]:
            sensor_spec = habitat_sim.SensorSpec()
            sensor_spec.uuid = sensor_uuid
            sensor_spec.sensor_type = sensor_params["sensor_type"]
            sensor_spec.resolution = sensor_params["resolution"]
            sensor_spec.position = sensor_params["position"]

            sensor_specs.append(sensor_spec)
            
    # Here you can specify the amount of displacement in a forward action and the turn angle
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs
    agent_cfg.action_space = {
        "move_forward": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "turn_left": habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=30.0)
        ),
        "turn_right": habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=30.0)
        ),
    }
    
    return habitat_sim.Configuration(sim_cfg, [agent_cfg])

In [5]:
cfg = make_cfg(sim_settings)
sim = habitat_sim.Simulator(cfg)


I0620 12:22:58.825727 18021 simulator.py:146] Loaded navmesh /media/shubodh/DATA/Downloads/data-non-onedrive/replica_v1/apartment_0/habitat/mesh_semantic.navmesh


In [6]:
cfg.agents[0].sensor_specifications[2].parameters

MapStringString{far: 1000, hfov: 90, near: 0.01}

# Scene semantic annotations

In [7]:
def print_scene_recur(scene, limit_output=10):
    print(f"House has {len(scene.levels)} levels, {len(scene.regions)} regions and {len(scene.objects)} objects")
    print(f"House center:{scene.aabb.center} dims:{scene.aabb.sizes}")
    
    count = 0
    for level in scene.levels:
        print(
            f"Level id:{level.id}, center:{level.aabb.center},"
            f" dims:{level.aabb.sizes}"
        )
        for region in level.regions:
            print(
                f"Region id:{region.id}, category:{region.category.name()},"
                f" center:{region.aabb.center}, dims:{region.aabb.sizes}"
            )
            for obj in region.objects:
                print(
                    f"Object id:{obj.id}, category:{obj.category.name()},"
                    f" center:{obj.aabb.center}, dims:{obj.aabb.sizes}"
                )
                count += 1
                if count >= limit_output:
                    return None

# Print semantic annotation information (id, category, bounding box details) 
# about levels, regions and objects in a hierarchical fashion
scene = sim.semantic_scene
print_scene_recur(scene)

House has 0 levels, 0 regions and 462 objects
House center:[0. 0. 0.] dims:[-inf -inf -inf]


In [8]:
# i=0
# file1 = open("samply.txt","a")
# while i < len(scene.objects): 
#     if scene.objects[i] and scene.objects[i].category is not None:
#         print(i)
#         L = (f"{scene.objects[i].id} {scene.objects[i].category.name()} \n")
#         file1.writelines(L)
#     i = i+1
# file1.close()
# scene.objects[2].category.name()

In [82]:
from PIL import Image
from habitat_sim.utils.common import d3_40_colors_rgb

def display_save_sample(rgb_obs, semantic_obs, depth_obs, obs_id, save=True, visualize=False, debug=False):
    # Save raw data
    if save:
        raw_data_folder = "./data_collection/apartment_0/raw_data/" 
        np.save(raw_data_folder + str(obs_id) + "_rgb",rgb_obs)
        np.save(raw_data_folder + str(obs_id) + "_instance-seg",semantic_obs)
        np.save(raw_data_folder + str(obs_id) + "_depth",depth_obs)
    
    if debug:
        depth_obs_cm_uint16 = (depth_obs*100).astype(np.uint16)
        semantic_obs_uint16 = semantic_obs.astype(np.uint16)
#         print(f"depth {(depth_obs_cm_uint16.dtype)}")
#         print(f"semantic obs {semantic_obs_uint16.dtype}")
#         print(f" {obs_id} semantic obs unique: {np.unique(semantic_obs)}, semantic obs dim {semantic_obs.shape}")
        print(f" {obs_id} depth obs unique: {np.unique(depth_obs_cm_uint16)}")
    
    rgb_img = Image.fromarray(rgb_obs, mode="RGBA")
    
    semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
    semantic_img.putpalette(d3_40_colors_rgb.flatten())
    semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
    semantic_img = semantic_img.convert("RGBA")
    
    depth_img = Image.fromarray((depth_obs / 10 * 255).astype(np.uint8), mode="L")

    # Save visualization data
    if save:
        viz_data_folder = "./data_collection/apartment_0/viz_data/"
        rgb_img.save(viz_data_folder + str(obs_id) + "_rgb.png")
        semantic_img.save(viz_data_folder + str(obs_id) + "_instance-seg.png")
        depth_img.save(viz_data_folder + str(obs_id) + "_depth.png")
        print()
    # visualize first 5 frames
    if visualize and (obs_id < 5):
        arr = [rgb_img, semantic_img, depth_img]
        titles = ['rgb', 'semantic', 'depth']
        plt.figure(figsize=(12 ,8))
        for i, data in enumerate(arr):
            ax = plt.subplot(1, 3, i+1)
            ax.axis('off')
            ax.set_title(titles[i])
            plt.imshow(data, cmap='gray')
        plt.show()
    


In [83]:
random.seed(sim_settings["seed"])
sim.seed(sim_settings["seed"])

# Set agent state
poses_json = "/media/shubodh/DATA/OneDrive/rrc_projects/2020/DeepGlobalRegistration-Navigation/habitat_all/habitat-sim/examples/data_collection/apartment_0/poses_run-replica-apartment_0.json"
with open(poses_json) as f:
    poses_data = json.load(f)
    
    position_array = [list(map(float,i)) for i in poses_data['position']]
    rotation_array = [list(map(float,i)) for i in poses_data['rotation']]

In [84]:
poses_path = {
    'position': np.array(position_array),
    'rotation': np.array(rotation_array)
}

agent = sim.initialize_agent(sim_settings["default_agent"])
agent_state = habitat_sim.AgentState()
agent_state.position = poses_path['position'][0]
agent_state.rotation = habitat_sim.utils.common.quat_from_coeffs(poses_path['rotation'][0])
agent.set_state(agent_state)

# Get agent state
agent_state = agent.get_state()
print("agent_state: position", agent_state.position, "rotation", (agent_state.rotation))

agent_state: position [ 0.284301 -1.37477   5.26595 ] rotation quaternion(1, 0, 0, 0)


# Take random actions and display sensor data

In [85]:
total_frames = 0
action_names = list(
    cfg.agents[
        sim_settings["default_agent"]
    ].action_space.keys()
)

images_for_video = []


max_frames = 10 #len(poses_path['position'])
while total_frames < max_frames:
#     action = random.choice(action_names)
#     print("action", action)
    agent_state.position = poses_path['position'][total_frames]
    agent_state.rotation = habitat_sim.utils.common.quat_from_coeffs(poses_path['rotation'][total_frames])
    agent.set_state(agent_state)
    
    observations = sim.get_sensor_observations()
#     print((observations))
    rgb = observations["color_sensor"]
    semantic = observations["semantic_sensor"]
    depth = observations["depth_sensor"]
    
#     print("Debug")
#     print(np.unique(semantic))
    
    images_for_video.append(rgb)
    
    display_save_sample(rgb, semantic, depth, total_frames, save=False, visualize=False, debug=True)
    
    agent_state = agent.get_state()
#     print("agent_state: position", agent_state.position, "rotation", agent_state.rotation)

    total_frames += 1

 0 depth obs unique: [100 118 119 120 121 122 123 124 126 127 128 129 130 131 132 134 135 136
 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154
 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172
 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190
 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208
 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226
 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244
 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262
 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280
 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298
 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316
 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334
 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352
 353 354 355 356 357 358 359 3

 3 depth obs unique: [100 101 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132
 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150
 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186
 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204
 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222
 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240
 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258
 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276
 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294
 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312
 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330
 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348
 349 350 351 352 353 354 355 3

 6 depth obs unique: [  0  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114
 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132
 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150
 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186
 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204
 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222
 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240
 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258
 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276
 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294
 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312
 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330
 331 332 333 334 335 336 337 3

 9 depth obs unique: [  0  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115
 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169
 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205
 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223
 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241
 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259
 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277
 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295
 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313
 314 315 316 317 318 319 320 3

In [81]:
import imageio
import tqdm
from habitat.core.logging import logger
from typing import Dict, List, Optional, Tuple


def img_to_video(images: List[np.ndarray],
                output_dir: str,
                video_name: str,
                fps: int = 30,
                quality: Optional[float] = 5,
                **kwargs,):
    
    assert 0 <= quality <= 10
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    video_name = video_name.replace(" ", "_").replace("\n", "_") + ".mp4"
    writer = imageio.get_writer(
        os.path.join(output_dir, video_name),
        fps=fps,
        quality=quality,
        **kwargs,
    )
    logger.info(f"Video created: {os.path.join(output_dir, video_name)}")
    for im in tqdm.tqdm(images):
        writer.append_data(im)
    writer.close()

In [ ]:
# img_to_video(images_for_video, './data_collection/apartment_0/', 'data_run_video')